# Omitted Variable Bias

This notebook displays the effect of Omitted Variable Bias. (https://en.wikipedia.org/wiki/Omitted-variable_bias)
<br>
Full model: $Y=\alpha+\beta_1 \cdot X+\beta_2 \cdot Z + \epsilon$ with $\alpha=0.5, \beta_1=2$.
<br>
Reduced model (omitting Z): $Y=\alpha+\beta_1 \cdot X + \epsilon$
<br><br>
Omitted Variable Bias occurs when 
<br>1) Z is correlated with X ($cov(Z, X) \neq 0$) 
<br>2) Z has a non-zero coefficient ($\beta_2 \neq 0$).

To create the visualization, click on the code block below, and run that block by clicking on the 'run cell' button marked with red on the following image.

![title](Menu.PNG)

Running the code cell for the first time may take a while. Once the run is complete, you'll see some slidebars/dropdown menus along with the 'Run Interact' button. Adjust the parameters, and then press the button to see the visualization.

Make sure the two parameters, beta2 and cov, are set to nonzero to see the effect of the bias.
<br>You can adjust the 'angle' and 'height' parameter to rotate & change the height of the 3d plot.

In [1]:
from visfunctions import OVB
from ipywidgets import interact_manual

interact_manual(OVB, beta2=(-5, 5), cov=(-0.9, 0.9), angle=(0, 360), height=(0, 40))

<function visfunctions.OVB>